# Sveltish
> A Python package that ~~kind of~~ implements Svelte Stores.


`Svelte Stores` are one of the secret weapons of the [Svelte framework](https://svelte.dev/) (the recently voted [most loved web framework](https://insights.stackoverflow.com/survey/2021#section-most-loved-dreaded-and-wanted-web-frameworks)). 

Stores allow easy [reactive programming](https://en.wikipedia.org/wiki/Reactive_programming) by presenting an [Observer pattern](https://en.wikipedia.org/wiki/Observer_pattern) that is as simple as necessary, but not simpler.

## Install

```sh
pip install sveltish
```

## How to use

Sometimes, you'll have values that need to be accessed by multiple unrelated objects.

For that, you can use `stores`.  It is a very simple implementation (around 100 lines of code) of the Observer/Observable pattern. 

A store is simply an object with a `subscribe` method that allows interested parties to be notified when its value changes. 

#### __Writable Stores__ 

In [ ]:
from sveltish.stores import Writable

In [ ]:
#|hide
from nbdev import show_doc
from fastcore.test import test_eq

In [ ]:
#|hide
show_doc(Writable)

---

[source](https://github.com/fredguth/sveltish/blob/main/sveltish/stores.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Writable

>      Writable (initial_value:Any=None, start:Notifier=<function <lambda>>)

A Writable Store.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| initial_value | Any | None | initial value of the store |
| start | Notifier | <lambda> | A Notifier (Optional) |
| **Returns** | **None** |  |  |

In [ ]:
count = Writable(0)
history = []  # logging for testing
# subscribe returns an unsubscriber
def record(x): 
    history.append(x)
    print(history)
stop = count.subscribe(record)
count

[0]


Writable(0)

We just created a store, `count`. Its value can be accessed via a `callback` we pass in the `count.subscribe` method:

A __Writable__ can be set from the outside. When it happens, all its subscribers will react.

In [ ]:
def increment(): count.update(lambda x: x + 1)
def decrement(): count.update(lambda x: x - 1)
def reset(): count.set(0)

count.set(3)
increment()
decrement()
decrement()
reset()
count.set(42)


test_eq(history, [0, 3, 4, 3, 2, 0, 42])

[0, 3]
[0, 3, 4]
[0, 3, 4, 3]
[0, 3, 4, 3, 2]
[0, 3, 4, 3, 2, 0]
[0, 3, 4, 3, 2, 0, 42]


The `unsubscriber`, in this example the `stop` function, stops the notifications to the `subscriber`. 

In [ ]:
stop()
reset()
count.set(22)
test_eq(history, [0, 3, 4, 3, 2, 0, 42])
count

Writable(22)

Notice that you can still change the `store` but there was no print message this time.  There was no observer listening. 

:::{.callout-note}
`Observer`, `Subscriber` and `Callback` are used as synomyms here.
:::

When we subscribe new callbacks, they will be promptly informed of the current state of the `store`.

In [ ]:
stop  = count.subscribe(lambda x: print(f"Count is now {x}"))
stop2 = count.subscribe(lambda x: print(f"double of count is {2*x}"))

Count is now 22
double of count is 44


In [ ]:
reset()

Count is now 0
double of count is 0


In [ ]:
stop()
stop2()

You can create an empty `Writable Store`.

In [ ]:
store = Writable()
history = []
unsubscribe = store.subscribe(lambda x: history.append(x))
unsubscribe()
test_eq(history, [None])

If you try to unsubscribe twice, it won't break.  It just does nothing the second time... and in the third time... and...

In [ ]:
unsubscribe(), unsubscribe(), unsubscribe()

(None, None, None)

Stores assume mutable objects. 

::: {.callout-note}
In Python everythong is an object.  Here we are calling an object something that is not a primitive (eg. int, bool, etc)
:::

In [ ]:
class Bunch:
    __init__ = lambda self, **kw: setattr(self, '__dict__', kw)

obj = Bunch()
called = 0
store = Writable(obj)
def callback(x):
    global called
    called += 1
stop = store.subscribe(callback)

In [ ]:
test_eq(called, 1)
obj.a = 1 #type: ignore
store.set(obj)
test_eq(called, 2)

#### __Readable Stores__

However... It is clear that not all stores should be writable by whoever has a reference to them. Many times you want a single `publisher` of change in store that is only consumed (`subscribed`) by many other objects. For those cases, we have readable stores.

:::{.callout-note}
The `Publisher Subscriber (PubSub)` pattern is a variant of the `Observable/Observer` pattern.
:::

In [ ]:
from sveltish.stores import Readable

In [ ]:
#|hide
show_doc(Readable)

---

[source](https://github.com/fredguth/sveltish/blob/main/sveltish/stores.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Readable

>      Readable (initial_value:T, start:Notifier)

A Readable Store.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| initial_value | T | initial value of the store |
| start | Notifier | function called when the first subscriber is added |
| **Returns** | **None** |  |

In [ ]:
# from sveltish.stores import Readable
# from threading import Event, Thread
# import time


# def start(set): # the start function is the publisher
#     stopped = Event()
#     def loop(): # needs to be in a separate thread
#         while not stopped.wait(1): # in seconds
#             set(time.localtime())
#     Thread(target=loop).start()    
#     return stopped.set
   
# now = Readable(time.localtime(), start)

:::{.callout-note}
The `loop` needs to be in its own thread, otherwise the function would never return and we would wait forever.
:::

In [ ]:
# now

Readable(time.struct_time(tm_year=2023, tm_mon=2, tm_mday=23, tm_hour=18, tm_min=25, tm_sec=36, tm_wday=3, tm_yday=54, tm_isdst=0))

While there is no subscriber, the Readable will not be updated.

In [ ]:
# now

Readable(time.struct_time(tm_year=2023, tm_mon=2, tm_mday=23, tm_hour=18, tm_min=25, tm_sec=36, tm_wday=3, tm_yday=54, tm_isdst=0))

In [ ]:

# OhPleaseStop = now.subscribe(lambda x: print(time.strftime(f"%H:%M:%S", x), end="\r"))


In [ ]:
# time.sleep(3)
# OhPleaseStop()

A Readable store without a `start` function is a constant value and has no meaning for us. Therefore, `start` is a required argument.

In [ ]:
try:
    c = Readable(0) # shoud fail
except Exception as error:
    print(error)

__init__() missing 1 required positional argument: 'start'


::: {.callout-note} The Svelte Store api allow you to create a Readable Store without a Notifier. See discussion [here.](https://github.com/sveltejs/svelte/issues/8300)

#### __Derived Stores__

A `Derived Store` stores a value based on the value of another store.

In [ ]:
from sveltish.stores import Derived

In [ ]:
#|hide
show_doc(Derived)

---

[source](https://github.com/fredguth/sveltish/blob/main/sveltish/stores.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Derived

>      Derived (s:Union[Store,list[Store]], fn:Callable)

A Derived Store.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| s | Union[Store, list[Store]] | source store(s) |
| fn | Callable | a callback that takes the source store(s) values and returns the derived value |
| **Returns** | **None** |  |

In [ ]:
count = Writable(1)
stopCount = count.subscribe(lambda x: print(f"count is {x}"))
double = Derived(count, lambda x: x * 2)
stopDouble = double.subscribe(lambda x: print(f"double is {x}"))
test_eq(double.get(), 2*count.get())

count is 1
double is 2


In [ ]:
count.set(2)
test_eq(double.get(), 4)

count is 2
double is 4


Building on our previous example, we can create a store that derives the elapsed time since the original store was started.

In [ ]:
# def calc_elapsed(then):
#     now = time.localtime()
#     return time.mktime(now) - time.mktime(then)

In [ ]:
# elapsed = Derived(now, calc_elapsed)

In [ ]:
# stopElapsed = elapsed.subscribe(lambda x: print(f"Elapsed time: {x} seconds.", end="\r"))

In [ ]:
# time.sleep(10)
# stopElapsed()

Derived stores allow us to transform the value of a store. In RxPy they are called `operators`. You can build several operators like: `filter`, `fold`, `map`, `zip`...

Let's build a custom `filter` operator:

In [ ]:
user = Writable({"name": "John", "age": 32})
stopLog = user.subscribe(lambda x: print(f"User: {x}"))

User: {'name': 'John', 'age': 32}


In [ ]:
name = Derived(user, lambda x: x["name"])
stopName = name.subscribe(lambda x: print(f"Name: {x}"))

Name: John


In [ ]:
user.update(lambda x: x | {"age": 45})

User: {'name': 'John', 'age': 45}


Updating the age does not trigger the name subscriber.

In [ ]:
user.update(lambda x: x | {"name": "Fred"})

Name: Fred
User: {'name': 'Fred', 'age': 45}


Only changes to the name of the user triggers the `name` subscriber.

Another cool thing about Derived Stores is that you can derive from a list of stores. Let's build a `zip` operator.

In [ ]:
a = Writable([1,2,3,4])
b = Writable([5,6,7,8])
a,b

(Writable([1, 2, 3, 4]), Writable([5, 6, 7, 8]))

In [ ]:
zipper = Derived([a,b], lambda a,b: list(zip(a,b)))
test_eq(zipper.get(), [(1, 5), (2, 6), (3, 7), (4, 8)])

In [ ]:
a.set([4,3,2,1])
test_eq(zipper.get(), [(4, 5), (3, 6), (2, 7), (1, 8)])